<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To check GPU
!nvidia-smi

Thu Dec  5 11:32:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-7pr99nv6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-7pr99nv6
  Resolved https://github.com/huggingface/transformers.git to commit beb2c66ec369135b4a41e8552948a0e7e073dceb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --upgrade torch datasets accelerate peft bitsandbytes trl

In [ ]:
import transformers
import torch
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Gated model which requires huggingface login
from huggingface_hub import login
from google.colab import userdata
access_token_read = userdata.get('huggingface_login') # Get access token from HuggingFace account
login(token = access_token_read)

In [ ]:
# Define the model name (Mistral 7B Instruct model)
model_name = 'mistralai/Mistral-7B-Instruct-v0.1'

# Load the model configuration from the pretrained model
model_config = transformers.AutoConfig.from_pretrained(model_name)

# Initialize the tokenizer with remote code trusted (for custom tokenizer implementations)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set the pad token to the end-of-sequence (EOS) token for better padding compatibility
tokenizer.pad_token = tokenizer.eos_token

# Specify that padding should be added on the right side (common in most NLP tasks)
tokenizer.padding_side = "right"

# Flag to indicate whether 4-bit quantization should be used for the model
use_4bit = True

# Define the computation data type for 4-bit quantization (float16 for reduced memory usage)
bnb_4bit_compute_dtype = "float16"

# Specify the quantization type; "nf4" stands for Normal Float 4 (improves precision for low-bit quantization)
bnb_4bit_quant_type = "nf4"

# Flag to control nested quantization (set to False to avoid additional quantization layers)
use_nested_quant = False

# Get the appropriate PyTorch data type (e.g., torch.float16) for computation
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Configure the BitsAndBytes (bnb) 4-bit quantization settings
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,                    # Enable 4-bit quantization
    bnb_4bit_quant_type=bnb_4bit_quant_type,  # Specify the quantization type (NF4)
    bnb_4bit_compute_dtype=compute_dtype,     # Set the computation data type (float16)
    bnb_4bit_use_double_quant=use_nested_quant # Use or disable double quantization (nested quantization)
)

# The configured tokenizer and quantization settings can now be used to load the model with optimized performance.

In [ ]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=2048,
    output_scores=True
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model-00001-of-00002.safetensors:  25%|##5       | 2.50G/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
